In [ ]:
import pandas as pd
import pickle
import xgboost as xgb

DMatrix (obiekt danych dla xgboost) z ramkami danych po feature selection.

In [ ]:
d_train = xgb.DMatrix('../data/dtest.buff')
d_test = xgb.DMatrix('../data/dtrain.buff')

In [ ]:
d_train.feature_names

### Trening

Dużo parametrów, które można dostosować.

In [ ]:
params = {'objective':'binary:logistic', 'eval_metric':'auc', 'verbosity':2, 'nthread':2,
          'alpha': 0.33444991443971983,
 'colsample_bylevel': 0.9032325336057312,
 'colsample_bynode': 0.956089310886066,
 'colsample_bytree': 0.9732573760585466,
 'eta': 0.05996574700591213,
 'gamma': 1.335882297998541,
 'lambda': 1.1967442963491903,
 'max_depth': 12,
 'subsample': 0.8919065011000327
          }
results = {}

In [ ]:
model = xgb.train(params, d_train, 250,evals=[(d_test, 'test')], evals_result=results)

F1 score dla zbioru testowego.

In [ ]:
from sklearn.metrics import f1_score
import numpy as np
idx = list()
scores = list()
for i in np.arange(0,0.3,0.001):
    idx.append(i)
    y_hat = (model.predict(d_test)>i).astype('int')
    scores.append(f1_score(d_test.get_label(), y_hat))
pd.DataFrame({'threshold':idx, 'f1':scores}).max()   

In [ ]:
import matplotlib.pyplot as plt
plt.plot(results['test']['auc'])
plt.title('Przebieg uczenia, AUC na testowym')

Feature importance, liczba splitów wg. danego ficzera.

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(16, 10))
xgb.plot_importance(model, ax= ax, max_num_features=15)

Feature importance, wg. metryki 'gain'.

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(16, 10))
xgb.plot_importance(model, ax= ax, max_num_features=15, importance_type='gain')

## Crosswalidacja

In [ ]:
history = xgb.cv(params=params, dtrain=d_train,num_boost_round=400,
                            nfold=4, metrics='auc',early_stopping_rounds=100,
                            seed=108, verbose_eval=False,show_stdv=True)

In [ ]:
fig, ax = plt.subplots()
ax.plot(history.index, history['train-auc-mean'], label='train-auc-mean')
ax.fill_between(history.index,
        history['train-auc-mean'] + history['train-auc-std'],
        history['train-auc-mean'] - history['train-auc-std'], alpha=0.5)
ax.plot(history.index, history['test-auc-mean'], label='test-auc-mean')
ax.fill_between(history.index,
        history['test-auc-mean'] + history['test-auc-std'],
        history['test-auc-mean'] - history['test-auc-std'], alpha=0.5)
fig.set_size_inches(12,8)
ax.legend()
ax.grid()

In [ ]:
history.loc[history.shape[0]-1, 'test-auc-mean']